In [3]:
import random

In [4]:
class Card:
    def __init__(self, level, color, cost, points):
        #int ranges from 1 to 3
        self.level = level
        #We would keep as int based on this mapping  {0: 'black', 1:'blue', 2: 'green', 3: 'red', 4:'white'}
        self.color = color
        #We do cost as an array [black, ...., white] [0, 2, 1, 0, 0]
        self.cost = cost
        #int, ranges from 0 to 5
        self.points = points

class Noble:
    def __init__(self, requirements, points):
        #We do requirements as an array [black, ...., white] [3, 3, 3, 0, 0]
        self.requirements = requirements
        #int ranges from 3 to 4
        self.points = points

class GemColor:
    def __init__(self, name):
        self.name = name

In [14]:
class SplendorAgent:
    def __init__(self, gem_reserve=None, card_supply=None, players_state=None, nobles=None):
        self.gem_reserve = gem_reserve if gem_reserve is not None else [0,0,0,0,0, 0]
        self.player_cards = card_supply if card_supply is not None else []
        self.player_reserved_cards = []
        self.nobles = nobles if nobles is not None else []
        self.buying_power = self.gem_reserve 
        
        #generate binary list
        # Generate all possible binary lists of length 5
        binary_lists = [[i, j, k, l, m] for i in range(2) for j in range(2) for k in range(2) for l in range(2) for m in range(2)]

        # Filter the binary lists to keep only those with a sum of 3
        self.binary_lists_with_sum_of_3 = [binary_list for binary_list in binary_lists if sum(binary_list) == 3]
        self.binary_lists_with_sum_of_2 = [binary_list for binary_list in binary_lists if sum(binary_list) == 2]


    def UpdateBuyingPower(self):
        self.buying_power = self.gem_reserve
        for card in self.player_cards:
            self.buying_power[card.color] +=1
        

    def get_legal_actions(self, spl_env):

        legal_actions = []

        #can buy card #TO BE PUT IN A NICE LOOKING FUNCTION
        for board in [spl_env.self.board_1, spl_env.self.board_2, spl_env.self.board_3]:
                card_buy =[]
                for card in board:
                    over = 0
                    for i in range(len(card.cost)):
                        over += max(0, card.cost[i]- self.buying_power[i])

                    if over <= self.buying_power[-1]:
                        card_buy.append(card)
        card_buy = [('buy card', x) for x in card_buy]
        legal_actions.extend(card_buy)

        #RESERVE #TO BE PUT IN A NICE LOOKING FUNCTION
        reserve_cards = []
        if spl_env.self.gems_av[-1] >0 and len(self.player_reserved_cards) <3:
            for board in [spl_env.self.board_1, spl_env.self.board_2, spl_env.self.board_3]:
                    for card in board:
                            reserve_cards.append(card)
        reserve_cards = [('reserve card', x) for x in reserve_cards]
        legal_actions.extend(reserve_cards)

        #Buy reserved cards
        buyable_reserved_cards = []
     
        for card in self.player_reserved_cards:
                    over = 0
                    for i in range(len(card.cost)):
                        over += max(0, card.cost[i]- self.buying_power[i])

                    if over <= self.buying_power[-1]:
                        buyable_reserved_cards.append(card)
        buyable_reserved_cards = [('buy reserved', x) for x in buyable_reserved_cards]
        legal_actions.extend(buyable_reserved_cards)

        #Take Gems

        #make the list of gems currently takable
        doable_3gems = []
        for comb in self.binary_lists_with_sum_of_3:
            if  min([spl_env.gem_av[i] - comb[i] for i in range(len(comb))]) >=0:
                doable_3gems.append(comb)

        doable_2gems = []
        for comb in self.binary_lists_with_sum_of_2:
            if  min([spl_env.gem_av[i] - comb[i] for i in range(len(comb))]) >=0:
                doable_2gems.append(comb)
            
        
        gem_buy= []
        #append the list of 3 gems if the player has already 7 gems or less
        if sum(self.gem_reserve) <= 7:
            gem_buy.extend(doable_3gems)

        #append the list of 3 gems if the player has already 8

        if sum(self.gem_reserve)  == 8:
            gem_buy.extend(doable_2gems)

        #Appends the duos
        for x in range(len(spl_env.gem_av[:5])):
                list_empty = [0,0,0,0,0]
                if spl_env.gem_av[x] >3:
                    list_empty[x] = 2
                    gem_buy.append(list_empty)
        gem_buy = [('take gem', x) for x in gem_buy]
        legal_actions.extend(gem_buy)



    def execute_action(self, action):
        # execute the specified action and update the state
        action_type, action_params = action
        if action_type == 'buy':
            card = action_params
            self.buy_card(card)
        elif action_type == 'reserve':
            card = action_params
            self.reserve_card(card)
        elif action_type == 'take':
            noble = action_params
            self.take_noble(noble)

    def buy_card(self, card):
        # update gem and card supply and player's state when a card is bought
        for color, count in card.cost.items():
            if count > self.players_state[0]['gems'][color] + self.players_state[0]['cards'][card.color]:
                diff = count - (self.players_state[0]['gems'][color] + self.players_state[0]['cards'][card.color])
                if self.gem_supply[color] >= diff:
                    self.gem_supply[color] -= diff
                    self.players_state[0]['gems'][color] += diff
                else:
                    self.players_state[0]['gems'][color] += self.gem_supply[color]
                    self.gem_supply[color] = 0
                    self.players_state[0]['cards'][card.color] += (diff - self.gem_supply[color])
        self.players_state[0]['cards'][card.color] += 1
        self.card_supply[card.level].remove(card)

        # give player a noble if they meet its requirements
        for noble in self.nobles:
            if self.can_take_noble(noble):
                self.players_state[0]['nobles'].append(noble)
                self.nobles.remove(noble)
                break

    def reserve_card(self, card):
        # update gem and card supply and player's state when a card is reserved
        self.players_state[0]['reserved_cards'].append(card)
        self.gem_supply[-1] -= 1
        self.card_supply[card.level].remove(card)

    def take_noble(self, noble):
        # update player's state when a noble is taken
        self.players_state[0]['nobles'].append(noble)
        self.nobles.remove(noble)

    def get_player_score(self, player_id):
        # calculate and return the score of the specified player
        score = self.players_state[player_id]['prestige_points']
        for noble in self.players_state[player_id]['nobles']:
            score += noble.points
        for color in range(len(self.players_state[player_id]['cards'])):
            score += self.players_state[player_id]['cards'][color] * self.card_points[color]
        return score

    def get_winner(self):
        # determine and return the winner
        max_score = -1
        winner = None
        for i in range(len(self.players_state)):
            player_score = self.get_player_score(i)
            if player_score > max_score:
                max_score = player_score
                winner = i
        return winner

In [17]:
class SplendorEnvironment:
    def __init__(self, deck, deck_nobles, initial_gems = [7,7,7,7,7,5]):


        self.gems_av= initial_gems
    
        #creates the card decks 
        self.deck_1 = deck[0]
        self.deck_2 = deck[1]
        self.deck_3 = deck[2]

        #creates the cardboard
        self.board_1 = []
        self.board_2 = []
        self.board_3 = []

        self.UpdateCardBoard()

        #creates the noble board
        self.nobles = []
        self.CreateNobles(deck_nobles)

        #Create the gems
        self.gems= initial_gems
        

    def UpdateCardBoard(self):
        random.shuffle(self.deck_1)
        random.shuffle(self.deck_2)
        random.shuffle(self.deck_3)

        #update cards rank 1
        while len(self.board_1) <4:
            self.board_1.append(self.deck_1.pop(0))
            if len(self.deck_1) ==0:
                break
        #update cards rank 2
        while len(self.board_2) <4:
            self.board_1.append(self.deck_2.pop(0))
            if len(self.deck_2) ==0:
                break
        #update cards rank 3
        while len(self.board_3) <4:
            self.board_1.append(self.deck_3.pop(0))
            if len(self.deck_3) ==0:
                break
    
    def CreateNobles(self, deck_nobles):

        random.shuffle(deck_nobles)

        #update cards rank 1
        while len(self.nobles) <4:
            self.nobles.append(deck_nobles.pop(0))
           
      



    def reset(self, gem_supply, card_supply, players_state, nobles):
        self.state = SplendorState(gem_supply, card_supply, players_state, nobles)
        self.current_player = 0
        for i in range(self.num_players):
            self.state.players_state.append({'gems': [0, 0, 0, 0, 0], 'cards': [0, 0, 0, 0, 0], 'nobles': [], 'reserved_cards': [], 'prestige_points': 0})
            for j in range(4):
                self.draw_card(i)

    def draw_card(self, player_id):
        # draw a card from the top of the deck and add it to the player's hand
        level = min(sum([len(x) for x in self.state.card_supply]), 4) - 1
        if len(self.state.card_supply[level]) == 0:
            self.state.card_supply[level] = self.state.card_supply[level + 1][::-1]
            self.state.card_supply[level + 1] = []
        card = self.state.card_supply[level].pop()
        self.state.players_state[player_id]['cards'][card.color] += 1

    def step(self, action):
        # execute the specified action and update the current player
        self.state.execute_action(action)
        self.current_player = (self.current_player + 1) % self.num_players

    def get_legal_actions(self):
        return self.state.get_legal_actions()

    def get_current_player(self):
        return self.current_player

    def get_state(self):
        return self.state

    def is_terminal(self):
        # check if the game is over (i.e., if any player has 15 prestige points or if the card and noble supplies are exhausted)
        if any([player['prestige_points'] >= 15 for player in self.state.players_state]):
            return True
        if sum([len(x) for x in self.state.card_supply]) == 0 and len(self.state.nobles) == 0:
            return True
        return False

    def get_reward(self):
        # calculate the reward for each player based on their score
        rewards = [0] * self.num_players
        max_score = max([self.state.get_player_score(i) for i in range(self.num_players)])
        for i in range(self.num_players):
            if self.state.get_player_score(i) == max_score:
                rewards[i] = 1
        return rewards

In [30]:
#mapping  {0: 'black', 1:'blue', 2: 'green', 3: 'red', 4:'white'}
#I LEFT IT, BUT THE NEXT ONE RUNS FOR REAL

# create cards rank 1
cards_1= [
card1_1 = Card(1, 2, [0,0,0,0,0], 1)
card1_2 = Card(1, 2, [0,0,0,0,0], 1)
card1_3 = Card(1, 2, [0,0,0,0,0], 1)
card1_4 = Card(1, 3, [0,0,0,0,0], 1)
card1_5 = Card(1, 1, [0,0,0,0,0], 1)
card1_6 = Card(1, 0, [0,0,0,0,0], 1)]

#create cards rank 2
cards_2=[
card2_1 = Card(2, 2, [0,0,0,0,0], 3)
card2_2 = Card(2, 2, [0,0,0,0,0], 3)
card2_3 = Card(2, 2, [0,0,0,0,0], 3)
card2_4 = Card(2, 3, [0,0,0,0,0], 3)
card2_5 = Card(2, 1, [0,0,0,0,0], 3)
card2_6 = Card(2, 0, [0,0,0,0,0], 3)]

#create cards rank3
cards_3=[
card3_1 = Card(3, 2, [0,0,0,0,0], 5)
card3_2 = Card(3, 2, [0,0,0,0,0], 5)
card3_3 = Card(3, 2, [0,0,0,0,0], 5)
card3_4 = Card(3, 3, [0,0,0,0,0], 5)
card3_5 = Card(3, 1, [0,0,0,0,0], 5)
card3_6 = Card(3, 0, [0,0,0,0,0], 5)]

# create nobles
nobles= [
noble1 = Noble([0,3,3,3, 0], 3)
noble2 = Noble([4, 4, 0,0,0], 3)
noble3 = Noble([3,3,3,0,0], 3)

noble11 = Noble([0,3,3,3, 0], 3)
noble22 = Noble([4, 4, 0,0,0], 3)
noble33 = Noble([3,3,3,0,0], 3)
]
# create SplendorState
card_supply = [cards_1, cards_2, cards_3]

gem_supply = [7, 7, 7, 7, 7, 5]
players_state = [{'gems': [0,0,0,0,0,0],#the gems the person has (the 5 initial colors + joker)
                  'cards': [], #arrays with the cards the person has
                  'nobles': [], #the nobles the person got
                  'reserved_cards': [], # the cards the person reserved
                  'prestige_points': 0}# the points the person has
                  ]


# Environment
env = SplendorEnvironment(num_players=2)
env.reset(gem_supply=gem_supply, card_supply=card_supply, players_state=players_state, nobles=nobles)

# check that all players have 4 cards in their hand
for i in range(env.num_players):
    assert sum(env.state.players_state[i]['cards']) == 4

# execute a random action and check that the current player has changed
current_player = env.current_player
legal_actions = env.get_legal_actions()
action = legal_actions[0]
env.step(action)
assert env.current_player != current_player

# check that the player who took the action now has one more card in their hand
player_id = (current_player + 1) % env.num_players
card_color = action['card_color']
assert env.state.players_state[player_id]['cards'][card_color] == 1 

IndexError: ignored

In [27]:
#mapping  {0: 'black', 1:'blue', 2: 'green', 3: 'red', 4:'white'}

# create cards rank 1
cards_1= [Card(1, 2, [0,0,0,0,0], 1),
          Card(1, 2, [0,0,0,0,0], 1),
          Card(1, 2, [0,0,0,0,0], 1),
          Card(1, 3, [0,0,0,0,0], 1),
          Card(1, 1, [0,0,0,0,0], 1),
          Card(1, 0, [0,0,0,0,0], 1)]

#create cards rank 2
cards_2=[Card(2, 2, [0,0,0,0,0], 3),
         Card(2, 2, [0,0,0,0,0], 3),
         Card(2, 2, [0,0,0,0,0], 3),
         Card(2, 3, [0,0,0,0,0], 3),
         Card(2, 1, [0,0,0,0,0], 3),
         Card(2, 0, [0,0,0,0,0], 3)]

#create cards rank3
cards_3=[Card(3, 2, [0,0,0,0,0], 5),
         Card(3, 2, [0,0,0,0,0], 5),
         Card(3, 2, [0,0,0,0,0], 5),
         Card(3, 3, [0,0,0,0,0], 5),
         Card(3, 1, [0,0,0,0,0], 5),
         Card(3, 0, [0,0,0,0,0], 5)]

# create nobles
nobles= [
Noble([0,3,3,3, 0], 3),
Noble([4, 4, 0,0,0], 3),
Noble([3,3,3,0,0], 3),
Noble([0,3,3,3, 0], 3),
Noble([4, 4, 0,0,0], 3),
Noble([3,3,3,0,0], 3)]
# create SplendorState
card_supply = [cards_1, cards_2, cards_3]

env = SplendorEnvironment(card_supply, nobles)
print(env.board_1)

[<__main__.Card object at 0x000001F2CD4BB970>, <__main__.Card object at 0x000001F2CD4BBDC0>, <__main__.Card object at 0x000001F2CD4BBB20>, <__main__.Card object at 0x000001F2CD4BBA90>, <__main__.Card object at 0x000001F2CD4BB7C0>, <__main__.Card object at 0x000001F2CB6E74F0>, <__main__.Card object at 0x000001F2CB6E7670>, <__main__.Card object at 0x000001F2CB266250>, <__main__.Card object at 0x000001F2CD4BBD00>, <__main__.Card object at 0x000001F2CB6E7D90>, <__main__.Card object at 0x000001F2CD4BB7F0>, <__main__.Card object at 0x000001F2CB6E7DC0>, <__main__.Card object at 0x000001F2CB266730>, <__main__.Card object at 0x000001F2CB6E7550>, <__main__.Card object at 0x000001F2CD4AFE80>, <__main__.Card object at 0x000001F2CB6E7D30>]
